In [ ]:
import pathlib

import astropy.coordinates as coord
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from tqdm.auto import tqdm, trange

from scipy.spatial import cKDTree
from sklearn.decomposition import IncrementalPCA
from sklearn.neighbors import KernelDensity

from joaquin.config import Config
from joaquin.data import JoaquinData
from joaquin.plot import simple_corner, phot_to_label, plot_hr_cmd

In [ ]:
conf = Config('../config.yml')

plot_path = conf.plot_path / 'neighborhoods'
plot_path.mkdir(exist_ok=True)

In [ ]:
# Random number generator, using seed from config file
rng = np.random.default_rng(seed=conf.seed)

In [ ]:
parent = JoaquinData.read(conf.parent_sample_cache_file)
parent = parent[np.all(np.isfinite(parent.X), axis=1)]

Define a global spectral mask based on the fraction of pixels over the full parent sample that are masked:

In [ ]:
# MAGIC NUMBER / TODO: 0.25 should probably be a config setting
global_spec_mask = (parent.spec_bad_masks.sum(axis=0) / len(parent)) > 0.25

### Select a subset of stars to use for defining the neighborhoods:

In [ ]:
# TODO: these numbers or the filter used here should be configurable
idx = np.argwhere(
    (parent.stars['SNR'] > 200) &   # Strict spectral S/N cut
    (parent.stars['ruwe'] < 1.2) &  # Strict RUWE cut
    (parent.stars['N_COMPONENTS'] == 1)  # No SB2's
).ravel()

# TODO: this number (size) should be configurable
idx = rng.choice(idx, size=16384, replace=False)
subset = parent[idx]

len(subset)

In [ ]:
# Spectroscopic HR diagram of the subset stars:
fig, ax = plt.subplots(figsize=(6, 6))

teff_logg_bins = (
    np.linspace(3000, 9000, 128),
    np.linspace(-0.5, 5.75, 128))
ax.hist2d(parent.stars['TEFF'],
          parent.stars['LOGG'],
          bins=teff_logg_bins,
          norm=mpl.colors.LogNorm(),
          cmap='Greys')

ax.plot(subset.stars['TEFF'],
        subset.stars['LOGG'],
        ls='none', marker='o', mew=0, ms=2.,
        color='tab:blue', alpha=0.75)

ax.set_xlim(teff_logg_bins[0].max(),
            teff_logg_bins[0].min())
ax.set_ylim(teff_logg_bins[1].max(),
            teff_logg_bins[1].min())

ax.set_xlabel('TEFF')
ax.set_ylabel('LOGG')

fig.tight_layout()
fig.savefig(plot_path / 'subset-logg-teff.png', dpi=200)

### Construct the neighborhood feature matrix for the neighborhood node sample:

This currently uses the spectra and a set of colors

In [ ]:
color_labels = [f'{phot_to_label[x1]}-{phot_to_label[x2]}'
                for x1, x2 in conf.neighborhood_color_names]
tmp = subset.mask_spec_pixels(global_spec_mask)
subset_X = tmp.get_neighborhood_X(conf.neighborhood_color_names)

In [ ]:
tmp = subset_X[:, -len(color_labels):]
fig, _, cb = simple_corner(
    tmp, 
    labels=color_labels, 
    color_by=subset.stars['LOGG'], vmin=0.5, vmax=5.5, 
    colorbar=True)
cb.ax.set_aspect(40)
fig.savefig(plot_path / 'neighborhood-colors.png', dpi=200)

Run PCA on the neighborhood node features and project the subset feature matrix onto the PCA basis:

In [ ]:
pca = IncrementalPCA(n_components=conf.neighborhood_pca_components,
                     batch_size=1024)  # HACK
projected_X = pca.fit_transform(subset_X)
projected_X /= pca.singular_values_

This hacky step removes extreme outliers - only run this after the cells above because it overwrites variables!!

In [ ]:
mean = np.mean(projected_X, axis=0)
std = np.std(projected_X, axis=0)
bad_mask = np.any(np.abs(projected_X - mean) > 5*std, axis=1)

neighborhood_node_X = subset_X[~bad_mask]
neighborhood_node_stars = subset.stars[~bad_mask]

pca = IncrementalPCA(n_components=conf.neighborhood_pca_components,
                     batch_size=1024)
node_projected_X = pca.fit_transform(neighborhood_node_X)
node_projected_X /= pca.singular_values_

In [ ]:
fig, axes = plt.subplots(pca.n_components_ // 2, 2,
                         figsize=(16, 12), sharex=True)

for i in range(pca.n_components):
    axes.flat[i].plot(pca.components_[i])
    
fig.tight_layout()

In [ ]:
np.cumsum(pca.explained_variance_ratio_)

### Plot the spectroscopic parameters, colored by PCA component

In [ ]:
fig, axes = plt.subplots(3, 3, 
                         figsize=(10, 10),
                         sharex=True, sharey=True)

for i in range(pca.n_components):
    ax = axes.flat[i]
    ax.scatter(neighborhood_node_stars['TEFF'],
               neighborhood_node_stars['LOGG'],
               c=node_projected_X[:, i], s=6)
    ax.text(teff_logg_bins[0].max() - 100, 
            teff_logg_bins[1].min() + 0.1,
            f'PCA feature {i}', va='top', ha='left')

for i in range(pca.n_components, len(axes.flat)):
    axes.flat[i].set_visible(False)
    
ax.set_xlim(teff_logg_bins[0].max(), 
            teff_logg_bins[0].min())
ax.set_ylim(teff_logg_bins[1].max(), 
            teff_logg_bins[1].min())

fig.tight_layout()
fig.savefig(plot_path / 'neighborhood-logg-teff-pca.png', dpi=200)

In [ ]:
things = {
    'TEFF': (3000, 6500),
    'LOGG': (0.5, 5.5),
    'M_H': (-2, 0.5)
}
for name, (vmin, vmax) in things.items():
    fig, axes, cb = simple_corner(
        node_projected_X, 
        color_by=neighborhood_node_stars[name],
        colorbar=True,
        vmin=vmin, vmax=vmax,
        labels=[f'PCA {i}' 
                for i in range(pca.n_components_)])
    cb.ax.set_aspect(40)
    axes.flat[0].set_title(f'color: {name}')
    
    fig.savefig(plot_path / f'neighborhood-pca-{name}.png', dpi=200)
    plt.close(fig)

### Now use the neighborhood sample to define the neighborhoods:

We do this by estimating the local density (in projected feature space)

In [ ]:
bw = np.mean(np.std(node_projected_X, axis=0)) * np.sqrt(node_projected_X.shape[1]) / 3
kde = KernelDensity(bandwidth=bw, kernel='epanechnikov')
_ = kde.fit(node_projected_X)
dens = kde.score_samples(node_projected_X)

Now construct the kdtree to find neighbors:

In [ ]:
tree = cKDTree(node_projected_X)

Setting k=2 below means that each row in `idxs` will be (self, neighbor)

In [ ]:
_, idxs = tree.query(node_projected_X, k=2)

TODO: change this so that we skip a star if it appears in a block!

In [ ]:
sort_idx = dens.argsort()[::-1]

this_size = int(conf.max_neighborhood_size * len(neighborhood_node_stars) / len(parent))

neighborhoods = []
blocks = []
for (i1, i2) in idxs[sort_idx]:
    # This relaxes the uniqueness requirement: we only skip now if a 
    # star appears in a block
    if (np.ravel(blocks) == i1).sum() >= 2:
        continue
    
    _, results = tree.query(node_projected_X[i1], k=this_size)
    neighborhoods.append(np.concatenate(([i1], results)))
    blocks.append(np.concatenate(([i1], results[:conf.block_size])))

len(neighborhoods)

In [ ]:
node_TEFF = [neighborhood_node_stars['TEFF'][idx[0]]
             for idx in neighborhoods]
node_LOGG = [neighborhood_node_stars['LOGG'][idx[0]]
             for idx in neighborhoods]

fig, ax = plt.subplots(figsize=(6, 6))

ax.scatter(node_TEFF, node_LOGG)

ax.set_xlim(teff_logg_bins[0].max(), 
            teff_logg_bins[0].min())
ax.set_ylim(teff_logg_bins[1].max(), 
            teff_logg_bins[1].min())

ax.set_title('neighborhood nodes')

ax.set_xlabel('TEFF')
ax.set_ylabel('LOGG')

fig.tight_layout()

This makes plots of all of the neighborhood stars:

In [ ]:
neighbor_plot_path = plot_path / 'neighborhoods'
neighbor_plot_path.mkdir(exist_ok=True)

for name in tqdm(neighbor_plot_path.rglob('*.png')):
    name.unlink()

for n, hood in enumerate(tqdm(neighborhoods)):
    fig = plot_hr_cmd(parent.stars, neighborhood_node_stars,
                      idx0=hood[0], other_idx=hood[1:])
    fig.tight_layout()
    fig.savefig(neighbor_plot_path / f'neighborhood-{conf.max_neighborhood_size}-{n:03d}.png', dpi=200)
    plt.close(fig)

### Apply to the full parent sample

In [ ]:
tmp = parent.mask_spec_pixels(global_spec_mask)
parent_X = tmp.get_neighborhood_X(conf.neighborhood_color_names)

In [ ]:
parent_projected_X = np.zeros((parent_X.shape[0], node_projected_X.shape[1]),
                               dtype=np.float32)

vals = np.linspace(0, parent_X.shape[0], 32).astype(int)
for i1, i2 in zip(vals[:-1], vals[1:]):
    if i2 >= parent_X.shape[0]-1:
        i2 = parent_X.shape[0]
        
    parent_projected_X[i1:i2] = pca.transform(parent_X[i1:i2])
    parent_projected_X[i1:i2] /= pca.singular_values_

# Remove sources that we don't want to appear in the train/test neighborhoods:
# TODO: this should probably *also* be a configurable thing...
parent_projected_X[
    (parent.stars['N_COMPONENTS'] != 1) |
    (parent.stars['ruwe'] > 1.4) |  # MAGIC NUMBER
    (parent.stars['SNR'] < 80)] = np.inf

In [ ]:
parent_tree = cKDTree(parent_projected_X)

In [ ]:
zone_idx = []
for i, nidx in enumerate(tqdm(neighborhoods)):
    dist, idx = parent_tree.query(node_projected_X[nidx[0]],
                                  k=conf.max_neighborhood_size)
    zone_idx.append(idx[dist.argsort()])

In [ ]:
# for n in trange(len(zone_idx)):
#     fig = plot_hr_cmd(parent_data.stars, parent_data.stars,
#                       idx0=zone_idx[n][0], other_idx=zone_idx[n][1:])
    
#     fig.tight_layout()
#     fig.savefig(neighbor_plot_path / f'parent-neighborhood-{neighborhood_size}-{n:03d}.png', dpi=200)
#     plt.close(fig)

In [ ]:
for size in 2**np.arange(13, 15+1):
    all_indices = []
    for n in range(len(zone_idx)):
        all_indices.append(zone_idx[n][:size])
    tmp = np.unique(np.ravel(all_indices)).shape[0] / parent_projected_X.shape[0]
    print(f"{tmp*100:.1f}% of stars end up in a neighborhood of size {size}")

In [ ]:
len(parent), parent_projected_X.shape

In [ ]:
filename = conf.output_path / f'good_parent_neighborhood_indices.npy'
np.save(filename, np.array(all_indices))